In [2]:
import pandas as pd

data_path = "data/"

## read worker

In [3]:
worker_data = pd.read_csv(data_path + "worker_quality.csv")
worker_data.head()

,worker_id,worker_quality
0,2392073,-1
1,2527235,73
2,2531356,73
3,2400287,75
4,2523170,71


In [10]:
worker = {}
for i in range(worker_data.shape[0]):
    worker[int(worker_data.iloc[i]["worker_id"])] \
        = int(worker_data.iloc[i]["worker_quality"])

In [11]:
len(worker)

1807

## read project

In [15]:
project_list = pd.read_csv(data_path + "project_list.csv", header=None)
project_list.head()

,0,1
0,2908089,99
1,2909696,105
2,2909401,127
3,2909378,74
4,2909457,36


project_info结构：
key为project_id，value为：
* sub_category：项目子类别
* category：项目大类
* entry_count：项目entry的数目，相当于子任务
* entry_ids：每个entry的id
* client_feedback：请求者的反馈分数
* average_score：所有entry的平均分
* total_awards：项目给出的报酬
* start_date：开始时间
* deadline：结束时间


In [20]:
import json


entry_path = data_path + "entry/"
project_path = data_path + "project/"

project_info = {}
for i in range(project_list.shape[0]):
    project_id, entry_count = int(project_list.iloc[i][0]), int(project_list.iloc[i][1])
    
    # 未发布任务的project跳过
    if entry_count <= 0:
        continue
    
    p_dict = {} # 单个项目的属性
    chosen_attr = ["sub_category", "category", "entry_count", \
                   "client_feedback", "average_score", "total_awards",\
                    "start_date", "deadline", "entry_ids"]
    with open(project_path + "project_%d.txt" % project_id, "r") as f:
        project_data = json.loads(f.readlines()[0])
        for attr in chosen_attr:
            p_dict[attr] = project_data[attr]

    project_info[project_id] = p_dict

len(project_info)

2489

## read entry

entry_info结构：key为entry_id，value为：
* project_id：所属的project id
* worker_id：完成这个任务的worker的id
* score：完成这个任务的worker得到的分数

In [ ]:
import json, os
entry_info = {}


entry_paths = os.listdir(entry_path)
for e_path in entry_paths:
    if not "txt" in e_path:
        continue
    p_id = int(e_path.split("_")[1])

    with open(entry_path + e_path, "r") as f:
        entry_datas = json.loads(f.readlines()[0])['results']
        for entry_data in entry_datas:
            e_dict = {"project_id": p_id}
            e_id = entry_data["id"]
            e_dict["worker_id"] = entry_data["author"]
            e_dict["score"] = entry_data["revisions"][-1]["score"]
            entry_info[e_id] = e_dict

len(entry_info)

486483